Tarea 2 - Inteligencia Artificial

Integrantes:
- Matias Gonzalez
- Matias Ramirez

In [ ]:
import random
import pandas as pd

# ---------- Definición de la Mesa ----------
class Mesa:
    def __init__(self):
        self.cartas = self.generar_baraja()
        self.comunes = []
        self.bote = 0

    @staticmethod
    def generar_baraja():
        valores = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        palos = ['h', 'd', 'c', 's']
        return [f"{valor}{palo}" for valor in valores for palo in palos]

    def repartir_mano(self):
        return [self.cartas.pop(random.randint(0, len(self.cartas) - 1)) for _ in range(2)]

    def repartir_comunes(self, num):
        self.comunes.extend([self.cartas.pop(random.randint(0, len(self.cartas) - 1)) for _ in range(num)])

# ---------- Definición del Jugador ----------
class Jugador:
    def __init__(self, nombre, fichas=1000, es_bot=False):
        self.nombre = nombre
        self.fichas = fichas
        self.mano = None
        self.es_bot = es_bot

    def recibir_mano(self, mano):
        """Asigna una mano al jugador."""
        self.mano = mano

    def decidir_accion(self, mesa, q, limite):
        if self.es_bot:
            return decidir_accion_bot(self, mesa, q, limite)
        else:
            # Lógica para el jugador humano (por simplificar, este se ha omitido aquí)
            pass

# ---------- Definición de la Ronda ----------
class Ronda:
    def __init__(self, mesa, q, limite):
        self.mesa = mesa
        self.jugadores = []
        self.apuestas = {}
        self.q = q
        self.limite = limite

    def agregar_jugador(self, jugador):
        self.jugadores.append(jugador)
        self.apuestas[jugador] = 0

    def apostar(self, jugador, cantidad):
        self.mesa.bote += cantidad
        self.apuestas[jugador] += cantidad

    def jugar(self):
        for jugador in self.jugadores:
            jugador.recibir_mano(self.mesa.repartir_mano())
        num_cartas_comunes = [0, 3, 1, 1]
        for i in range(4):
            self.mesa.repartir_comunes(num_cartas_comunes[i])
            for jugador in self.jugadores:
                accion = jugador.decidir_accion(self.mesa, self.q, self.limite)
                if accion == "fold":
                    self.jugadores.remove(jugador)
                elif accion == "all-in":
                    self.apostar(jugador, jugador.fichas)
                    jugador.fichas = 0
                else:
                    self.apostar(jugador, accion)
                    jugador.fichas -= accion
        # Lógica para determinar el ganador (simplificada)
        ganador = random.choice(self.jugadores)  # Aquí se ha simplificado eligiendo al azar
        ganador.fichas += self.mesa.bote

# ---------- Funciones auxiliares ----------
def obtener_probabilidad_pre_flop(mano):
    mano_str = ''.join(sorted([card[0] for card in mano]))
    if mano[0][0] == mano[1][0]:  # Es un par
        matched_rows = df_probabilidades[df_probabilidades['Cards'] == mano_str]
    else:
        if mano[0][1] == mano[1][1]:  # Las cartas son del mismo palo
            mano_str += 's'
            matched_rows = df_probabilidades[df_probabilidades['Cards'] == mano_str]
        else:
            mano_str += 'o'
            matched_rows = df_probabilidades[df_probabilidades['Cards'] == mano_str]

    if matched_rows.empty:
        print(f"No se encontró la mano: {mano_str} en el DataFrame")
        return 0.0

    probabilidad = matched_rows['Rank'].values[0]
    probabilidad /= 169.0
    return probabilidad

def decidir_accion_bot(jugador, mesa, q, limite):
    probabilidad = obtener_probabilidad_pre_flop(jugador.mano) if len(mesa.comunes) == 0 else 0.5
    if random.random() < q:  # Estrategia probabilística
        if probabilidad > 0.8:
            return min(limite * 0.75, jugador.fichas)
        elif probabilidad < 0.1:
            return "fold"
        else:
            return min(limite * 0.5, jugador.fichas)
    else:  # Estrategia mentirosa
        return min(limite, jugador.fichas)

# ---------- Juego principal ----------
def jugar_poker():
    mesa = Mesa()
    q = 0.5
    limite = 100
    jugador1 = Jugador("Jugador1", es_bot=True)
    jugador2 = Jugador("Jugador2", es_bot=True)
    ronda = Ronda(mesa, q, limite)
    ronda.agregar_jugador(jugador1)
    ronda.agregar_jugador(jugador2)
    ronda.jugar()
    print(f"Fichas de {jugador1.nombre}: {jugador1.fichas}")
    print(f"Fichas de {jugador2.nombre}: {jugador2.fichas}")

# Carga la matriz de probabilidades
columnas = ["Rank", "Cards", "Type"]
df_probabilidades = pd.read_csv('https://raw.githubusercontent.com/BrenoCPimenta/Poker-preflop-hand-rank-scraping-to-csv/master/preFlop-rank.csv', header=None, names=columnas)

# Ejecuta el juego
jugar_poker()


No se encontró la mano: 29o en el DataFrame
Fichas de Jugador1: 1000
Fichas de Jugador2: 1000.0
